> <p><small><small>This Notebook is made available subject to the licence and terms set out in the <a href = "http://www.github.com/google-deepmind/ai-foundations">AI Research Foundations Github README file</a>.

![](https://storage.googleapis.com/dm-educational/assets/ai_foundations/GDM-Labs-banner-image-C2-white-bg.png)

# Lab: Implement a BPE Tokenizer


<a href='https://colab.research.google.com/github/google-deepmind/ai-foundations/blob/master/course_2/gdm_lab_2_4_implement_a_bpe_tokenizer.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

Implement a subword tokenizer using the byte pair encoding (BPE) algorithm.

25 minutes

## Overview

In this lab, you will implement the **byte pair encoding (BPE) algorithm** that was presented in the previous activity. This will provide you with a  more sophisticated tokenizer than the space tokenizer that you have been using in previous labs. This in return will allow you to train a better model using the Africa Galore dataset.

### What you will learn

By the end of this lab, you will understand:

* The input and output of the byte pair tokenization (BPE) algorithm.
* The iterative steps involved in learning a tokenizer.
* How to implement and run the full BPE algorithm over multiple merge steps.


### Tasks

In this lab, you will:

* Write code to convert each word into a list of characters with `</w>` appended.
* Complete the function `get_pair_frequencies` to count adjacent symbol pairs.
* Fill in logic in `merge_pair_in_word` to replace the most frequent pair.
* Apply merges iteratively to update the corpus and vocabulary.
* Implement the loop in `learn_bpe` to perform a fixed number of merge steps and inspect results.


## How to use Google Colaboratory (Colab)

Google Colaboratory (also known as Google Colab) is a platform that allows you to run Python code in your browser. The code is written in **cells** that are executed on a remote server.

To run a cell, hover over the cell and click on the `run` button to its left. The run button is the circle with the triangle (▶). Alternatively, you can also click on a cell and use the keyboard combination Ctrl+Return (or ⌘+Return if you are using a Mac).

To try this out, run the following cell. This should print today's day of the week below it.

In [ ]:
from datetime import datetime

print(f"Today is {datetime.today():%A}.")

Note that the *order in which you run the cells matters*. When you are working through a lab, make sure to always run *all* cells in order, otherwise the code might not work. If you take a break while working on a lab, Colab may disconnect you and in that case, you have to execute all cells again before  continuing your work. To make this easier, you can select the cell you are currently working on and then choose __Runtime → Run before__  from the menu above (or use the keyboard combination Ctrl/⌘ + F8). This will re-execute all cells before the current one.

## Imports

In [1]:
from collections import Counter, defaultdict # For counting token pairs.
import pandas as pd # For loading the dataset.

## Byte pair encoding (BPE)

Recall from the previous article that the BPE algorithm consists of the following steps:

1. **Initialization**: The entire dataset (a text corpus) is split into individual characters and the vocabulary is initialized with the set of unique characters. Spaces are replaced with a special end-of-word symbol (e.g., `</w>`).
2. **Counting**: For each adjacent pair of tokens, count how often each pair appears in your corpus.
3. **Merging**: Choose the adjacent pair of tokens `(p, q)` that appears most frequently in the corpus and add a new merged token `pq` to the vocabulary.
4. **Replacing**: Replace all adjacent pairs of tokens `(p, q)` with the new token `pq` in the corpus.
5. **Repeating**: Repeat steps 2-4 until you reach the target vocabulary size.

### Coding Activity 1: Initialization

You will use the Africa Galore dataset to perform one step of the BPE algorithm before running the algorithm for multiple iterations on a smaller corpus.

Run the following cell to load the dataset.

In [2]:
africa_galore = pd.read_json(
    "https://storage.googleapis.com/dm-educational/assets/ai_foundations/africa_galore.json"
)
dataset = africa_galore["description"].values
print(f"Dataset contains {dataset.shape[0]:,} paragraphs.")

Dataset contains 232 paragraphs.


The initialization step consists of three parts:
1. Converting every space-separated word into a list of characters.
2. Appending the special end-of-word symbol `"</w>"` at the end of each word's list of characters.
3. Adding the set of unique characters to your vocabulary.

<br />

-----
> **💻 Your task:**
>
> Complete the `bpe_initialize` function below such that `corpus` contains a list with each word in the dataset represented as a list of characters and the end-of-word symbol.
>
> Hint: You can turn a string `s` into a list of characters with the `list` function: `list(s)`.

-----

In [3]:
EOW_SYMBOL = "</w>"

def bpe_initialize(dataset: list[str]) -> tuple[list[list[str]], set[str]]:
    """Implements the initialization step of the byte pair encoding algorithm.

    Args:
      dataset: The corpus consisting of a list of raw paragraphs.

    Returns:
      corpus: A list containing every word in the corpus represented as a list
        of characters and the special end-of-word-symbol "</w>".
      vocabulary: The set of unique tokens in the dataset that serves as the
        vocabulary before the first merge.
    """

    corpus = []
    vocabulary = {EOW_SYMBOL}
    for paragraph in dataset:
        for word in paragraph.split(" "):
            # Convert word into chars and add end-of-word symbol ('</w>').
            chars =  list(word) + [EOW_SYMBOL]
            corpus.append(chars)

            # Append characters to the vocabulary. Since vocabulary is a set,
            # it will ignore characters that have already been added.
            vocabulary.update(chars)

    return corpus, vocabulary

corpus, vocabulary = bpe_initialize(dataset)

# Print the first 10 "words" in the corpus.
print("First 10 words: ")
for word in corpus[:10]:
    print(f"  {word}")

print("\n\nVocabulary:")
print(f"  {vocabulary}")

First 10 words: 
  ['T', 'h', 'e', '</w>']
  ['L', 'a', 'g', 'o', 's', '</w>']
  ['a', 'i', 'r', '</w>']
  ['w', 'a', 's', '</w>']
  ['t', 'h', 'i', 'c', 'k', '</w>']
  ['w', 'i', 't', 'h', '</w>']
  ['h', 'u', 'm', 'i', 'd', 'i', 't', 'y', ',', '</w>']
  ['b', 'u', 't', '</w>']
  ['t', 'h', 'e', '</w>']
  ['e', 'n', 'e', 'r', 'g', 'y', '</w>']


Vocabulary:
  {'u', 'F', '3', '—', 'M', '0', 'K', 'c', 'X', 'v', ';', 'P', 'W', 'z', 'f', 'I', 'H', 'p', 'N', '"', 'U', 't', 'd', 'g', ',', 's', 'j', '°', '“', '(', ':', '</w>', 'Z', '5', '?', 'm', 'h', '9', '.', 'y', 'A', 'D', 'E', 'r', 'e', '/', 'L', 'n', ')', 'x', 'é', 'k', '4', 'o', 'V', "'", '6', 'B', '-', '”', 'q', 'w', 'C', 'l', 'G', 'b', 'a', '7', 'T', '8', 'R', '1', 'J', 'O', '2', 'i', 'S', 'Y'}



### Coding Activity 2: Counting adjacent token pairs

The second step of the BPE algorithm is to count adjacent token pairs. For every word in the corpus, you have to extract every pair of tokens that appear next to each other and increase their count. For example, when encountering the word represented as `["r", "i", "c", "e", "</w">]`, the counts of the following four pairs should be incremented by 1: `(r,i)`, `(i,c)`, `(c,e)`, `(e, </w>)`.

<br />

-------
> **💻 Your task:**
>
> Complete the `get_pair_frequencies` function below.
>
> This function should return a [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) object whose keys are pairs of tokens and the value is the frequency of the pair in the corpus.
>
------

In [4]:
def get_pair_frequencies(
    corpus: list[list[str]]
) -> Counter[tuple[str, str], int]:
    """
    Calculates the frequency of adjacent character pairs in a corpus.

    Args:
      corpus: A list of tokenized words, where each word is represented as a
        list of subword tokens. Before the first merge these are all individual
        characters.

    Returns:
      A Counter object mapping each adjacent character pair (a tuple) to its
        frequency in the corpus.
    """
    pairs = Counter()
    for word in corpus:
        # Loop through the position of every token but the last one.
        for i in range(len(word) - 1):
            # Create a tuple representing the adjacent pair consisting of the
            # i-th and (i+1)-th token in `word`.
            pair =  (word[i], word[i+1])
            pairs[pair] += 1
    return pairs


pair_freqs = get_pair_frequencies(corpus)
print("The 10 most common pair frequencies are:")
for freq in pair_freqs.most_common(10):
    print(f" {freq}")

The 10 most common pair frequencies are:
 (('e', '</w>'), 2639)
 (('d', '</w>'), 2146)
 (('s', '</w>'), 2078)
 (('a', 'n'), 1883)
 (('t', 'h'), 1869)
 (('i', 'n'), 1822)
 (('h', 'e'), 1735)
 ((',', '</w>'), 1710)
 (('e', 'r'), 1359)
 (('n', 'd'), 1305)


The output shows the most frequent adjacent character pairs and their counts in the corpus. The pairs with `</w>` indicate characters that frequently appear at the end of words. For example, `("e", "</w>")` is the most frequent pair, meaning that many words in the corpus end with the letter "e". Other common pairs like `("i", "n")`, `("a", "n")`, and `("t", "h")` represent frequent character combinations within words. This information is used in the next step of the BPE algorithm to merge the most frequent pairs into new subword tokens.

### Coding Activity 3: Merge the most frequent pair

The third step of the BPE algorithm is to identify the most frequent pair of adjacent tokens and to merge them into a new subword token.

Run the following cell to identify the most frequent token pair and add it to the vocabulary.

In [5]:
most_freq_pair, freq = pair_freqs.most_common(1)[0]
print(
    f"The most frequent pair is {most_freq_pair} with a frequency of {freq:,}."
)

new_token = most_freq_pair[0] + most_freq_pair[1]
vocabulary.add(new_token)

The most frequent pair is ('e', '</w>') with a frequency of 2,639.


------
> **💻 Your task:**
>
> Complete the `merge_pair_in_word` function below.
>
> This function should replace all instances of `pair_to_merge`
> in `word` with a new subword token that consists of the two
> tokens in `pair_to_merge`.
------

In [9]:
def merge_pair_in_word(
    word: list[str], pair_to_merge: tuple[str, str]
) -> list[str]:
    """Merges adjacent occurrences of a specified pair of characters in a word.

    Given a word represented as a list of subword tokens and a pair of tokens to
    merge (represented as a tuple), this function returns a new list where every
    instance of the specified pair appearing adjacently in the original word is
    replaced by a single string representing the merged pair.

    Args:
      tokens: A list of subword tokens representing one space separated word.
      pair_to_merge: A pair of two subword tokens that should be merged into one
        subword token.

    Returns:
      New list of subword tokens representing the word after applying the merge.
    """
    merged_symbol = pair_to_merge[0] + pair_to_merge[1]
    i = 0
    new_word = []
    while i < len(word):
        # If this position and the next match the pair, merge them.
        if i < len(word) - 1 and (word[i],word[i+1]) == pair_to_merge:
            new_word.append(merged_symbol)
            i += 2
        else:
            new_word.append(word[i])
            i += 1
    return new_word

print(f"Before merging: {' '.join(corpus[0])}")
new_word = merge_pair_in_word(corpus[0], most_freq_pair)
print(f"After merging:  {' '.join(new_word)}")

Before merging: T h e </w>
After merging:  T h e</w>


### Replace tokens in corpus

Once the new token has been added to the vocabulary, you can use the `merge_pair_in_word` function that you implemented above to replace every occurrence of the most frequent pair with the new merged subword token. This is the fourth step of the BPE algorithm.

Run the following cell to perform the merges in the corpus.


In [10]:
new_corpus = []
for word in corpus:
    new_word = merge_pair_in_word(word, most_freq_pair)
    new_corpus.append(new_word)

# Each iteration of BPE results in a new corpus with merged pairs.
print(
    "The first 10 \"words\" in the corpus, with \"e</w>\" being a single token"
    " now:"
)
for word in new_corpus[:10]:
    print(f"  {word}")

print("\n\nVocabulary:")
print(f"  {vocabulary}")

The first 10 "words" in the corpus, with "e</w>" being a single token now:
  ['T', 'h', 'e</w>']
  ['L', 'a', 'g', 'o', 's', '</w>']
  ['a', 'i', 'r', '</w>']
  ['w', 'a', 's', '</w>']
  ['t', 'h', 'i', 'c', 'k', '</w>']
  ['w', 'i', 't', 'h', '</w>']
  ['h', 'u', 'm', 'i', 'd', 'i', 't', 'y', ',', '</w>']
  ['b', 'u', 't', '</w>']
  ['t', 'h', 'e</w>']
  ['e', 'n', 'e', 'r', 'g', 'y', '</w>']


Vocabulary:
  {'u', 'F', '3', '—', 'M', '0', 'K', 'c', 'X', 'v', ';', 'P', 'W', 'z', 'f', 'I', 'H', 'p', 'N', '"', 'U', 't', 'd', 'g', ',', 's', 'j', '°', '“', '(', ':', '</w>', 'Z', '5', '?', 'm', 'h', '9', '.', 'y', 'A', 'D', 'E', 'r', 'e', '/', 'L', 'n', ')', 'x', 'é', 'k', '4', 'o', 'V', "'", '6', 'B', '-', '”', 'q', 'w', 'C', 'l', 'G', 'b', 'a', '7', 'T', '8', 'R', '1', 'J', 'O', '2', 'i', 'S', 'Y', 'e</w>'}


Note how both instances of the word "the" are now split into three subword tokens because the final "e" has been merged with the end-of-word token.

### Repeat

The steps 2, 3, and 4 can now be repeated for either a fixed number of steps or until you reach a target vocabulary size.

On each iteration, exactly one pair of tokens is merged resulting in one more subword token in the vocabulary.

The function below implements the complete BPE algorithm that runs for a fixed number of `num_merges`.

In [11]:
def learn_bpe(
    dataset: list[str], num_merges: int
) -> tuple[list[tuple[str, str]], list[list[str]]]:
    """
    Learns byte pair encoding (BPE) merge operations from a dataset.

    This function implements the basic BPE algorithm, iteratively merging the
    most frequent adjacent character pairs in a corpus to create subword units.
    It continues merging until the specified number of merges is reached or all
    words are represented by a single token.

    Args:
      dataset: A list of paragraphs, where each sentence is a string.
      num_merges: The desired number of BPE merge operations to perform.

    Returns:
      merges: A list of tuples, where each tuple represents a merged pair of
        tokens. (e.g., [('a', 'b'), ('ab', 'c')]).  The order of tuples reflects
        the order in which merges were performed.
      vocabulary: The final vocabulary of all subword tokens.
      corpus: The final corpus after BPE is applied.  Each sentence is
        represented as a list of words, and each word is represented as a list
        of subword tokens.
    """
    # Step 1: Initialize corpus as list of lists of characters + </w>.
    corpus, vocabulary = bpe_initialize(dataset)

    merges = []
    for _ in range(num_merges):
        # Step 2: Count all adjacent-pair frequencies.
        pair_freqs = get_pair_frequencies(corpus)
        if not pair_freqs:
            break

        # Step 3: Pick the most frequent pair.
        most_freq_pair, freq = pair_freqs.most_common(1)[0]
        if freq < 1:
            break  # No more pairs to merge.
        merges.append(most_freq_pair)
        new_token = most_freq_pair[0] + most_freq_pair[1]
        vocabulary.add(new_token)

        # Step 4: Merge that pair in every word of the corpus.
        new_corpus = []
        for word in corpus:
            new_word = merge_pair_in_word(word, most_freq_pair)
            new_corpus.append(new_word)
        corpus = new_corpus

    # Return the list of merges, the vocabulary, and the final tokenized corpus.
    return merges, vocabulary, corpus

### Run BPE on a small corpus

Run the following cell to apply your BPE algorithm to a small collection of words.

In [12]:
sample_text = [
        "desert",
        "deserted",
        "deserts",
        "desert",
        "tested",
        "test",
        "deserted",
        "desert",
        "desertion",
        "desertion",
        "function",
]
# Learn BPE tokenizer with 10 merge operations.
merges, vocabulary, tokenized_corpus = learn_bpe(sample_text, num_merges=10)

print("Learned merges (in order):")
for i, pair in enumerate(merges, start=1):
    print(f" {i}. Merge {pair[0]!r} + {pair[1]!r}  →  {pair[0]+pair[1]!r}")

print("\nFinal tokenized corpus:")
for orig, tokenized in zip(sample_text, tokenized_corpus):
    print(f"  {orig:8s} → {' '.join(tokenized)}")

Learned merges (in order):
 1. Merge 'e' + 's'  →  'es'
 2. Merge 'd' + 'es'  →  'des'
 3. Merge 'des' + 'e'  →  'dese'
 4. Merge 'dese' + 'r'  →  'deser'
 5. Merge 'deser' + 't'  →  'desert'
 6. Merge 'desert' + '</w>'  →  'desert</w>'
 7. Merge 'e' + 'd'  →  'ed'
 8. Merge 'ed' + '</w>'  →  'ed</w>'
 9. Merge 'i' + 'o'  →  'io'
 10. Merge 'io' + 'n'  →  'ion'

Final tokenized corpus:
  desert   → desert</w>
  deserted → desert ed</w>
  deserts  → desert s </w>
  desert   → desert</w>
  tested   → t es t ed</w>
  test     → t es t </w>
  deserted → desert ed</w>
  desert   → desert</w>
  desertion → desert ion </w>
  desertion → desert ion </w>
  function → f u n c t ion </w>


You have now successfully trained your own BPE tokenizer on a small corpus. In later labs, you will incorporate this algorithm into a class which can tokenize any given text.

## Summary

In this lab, you implemented the **byte pair encoding (BPE) algorithm**, a cornerstone of modern text tokenization. Through this implementation, you have observed how BPE iteratively **merges** frequent character pairs to create **subword tokens**. This process is crucial for handling the vast and dynamic nature of human language. It is especially important in the context of **large vocabulary sizes** and the need to represent **out-of-vocabulary tokens**. Understanding BPE and other subword tokenization techniques is critical for anyone working with language models. It directly impacts model performance, efficiency, and the ability to handle diverse text data.

In the next module of this course, you will explore how language models encode meaning in a high-dimensional vector space.

## Solutions

The following cells provide reference solutions to the coding activities in this notebook. If you really get stuck after trying to solve the activities yourself, you may want to consult these solutions.

It is recommended that you *only* look at the solutions after you have tried to solve the activities *multiple times*. The best way to learn challenging concepts in computer science and artificial intelligence is to debug your code piece-by-piece until it works, rather than copying existing solutions.

If you feel stuck, you may want to first try to debug your code. For example, by adding additional print statements to see what your code is doing at every step. This will provide you with a much deeper understanding of the code and the materials. It will also provide you with practice on how to solve challenging coding problems beyond this course.

To view the solutions for an activity, click on the arrow to the left of the activity name. If you consult the solutions, do not copy and paste them into the cells above. Instead, look at them, and type them manually into the cell. This will help you understand where you went wrong.


### Coding Activity 1

In [ ]:
# Complete implementation of the bpe_initialize function.
def bpe_initialize(dataset: list[str]) -> tuple[list[list[str]], set[str]]:
    """Implements the initialization step of the byte pair encoding algorithm.

    Args:
      dataset: The corpus consisting of a list of raw paragraphs.

    Returns:
      corpus: A list containing every word in the corpus represented as a list
        of characters and the special end-of-word-symbol "</w>".
      vocabulary: The set of unique tokens in the dataset that serves as the
        vocabulary before the first merge.
    """

    corpus = []
    vocabulary = {EOW_SYMBOL}
    for paragraph in dataset:
        for word in paragraph.split(" "):
            # Convert word into chars and add end-of-word symbol ('</w>').
            chars = list(word) + [EOW_SYMBOL]
            corpus.append(chars)
            # Add all characters to the vocabulary. Since vocabulary is a set,
            # it will ignore characters that have already been added.
            vocabulary.update(chars)

    return corpus, vocabulary

### Coding Activity 2

In [ ]:
# Complete implementation of the get_pair_frequencies function.
def get_pair_frequencies(
    corpus: list[list[str]]
) -> Counter[tuple[str, str], int]:
    """Calculates the frequency of adjacent character pairs in a corpus.

    Args:
      corpus: A list of tokenized words, where each word is represented as a
        list of subword tokens. Before the first merge these are all individual
        characters.

    Returns:
      A Counter object mapping each adjacent character pair (a tuple) to its
        frequency in the corpus.
    """
    pairs = Counter()
    for word in corpus:
        # Loop through the position of every token but the last one.
        for i in range(len(word) - 1):
            # Create a tuple representing the adjacent pair consisting of the
            # i-th and (i+1)-th token in `word`.
            pair = (word[i], word[i+1])
            pairs[pair] += 1
    return pairs


### Coding Activity 3

In [ ]:
# Complete implementation of the merge_pair_in_word function.
def merge_pair_in_word(word: list[str], pair_to_merge: tuple[str, str]) -> list[str]:
    """Merges adjacent occurrences of a specified pair of characters in a word.

    Given a word represented as a list of subword tokens and a pair of tokens to
    merge (represented as a tuple), this function returns a new list where every
    instance of the specified pair appearing adjacently in the original word is
    replaced by a single string representing the merged pair.

    Args:
      tokens: A list of subword tokens representing one space separated word.
      pair_to_merge: A pair of two subword tokens that should be merged into one
        subword token.

    Returns:
      New list of subword tokens representing the word after applying the merge.

    """
    merged_symbol = pair_to_merge[0] + pair_to_merge[1]
    i = 0
    new_word = []
    while i < len(word):
        # If this position and the next match the pair, merge them.
        if i < len(word) - 1 and (word[i], word[i + 1]) == pair_to_merge:
            new_word.append(merged_symbol)
            i += 2
        else:
            new_word.append(word[i])
            i += 1
    return new_word